## Import modules

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten, Dense
import matplotlib.pyplot as plt
import csv
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

## Work with the Data
* Change the data into the required format
* Augment the data slightly
* Split the data into train and validation sets

### CHANGING INTO REQUIRED FORMAT

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
#Converts panda dataframe into numpy arrays
train = train.to_numpy() 
test = test.to_numpy()

In [ ]:
y_train = train[:, 0] #Seperate all labels in train set
x_train = train[:, 1:] #Seperate all pixel values in train set
x_test = test[:, :] #Get all pixel values for test set
x_train = np.reshape(x_train, (-1, 28, 28, 1))
x_test = np.reshape(x_test, (-1, 28, 28, 1))
print(y_train.shape, x_train.shape, x_test.shape)

### AUGMENT THE DATA

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255., 
                             rotation_range = 15,
                             height_shift_range = 0.15,
                             width_shift_range = 0.15,
                             zoom_range = 0.15)

### SPLIT THE DATA

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

## Create the model
### Train two slightly different models simultaneously:
#### First model contains maxpooling layer after convolution layer, Second model contains convolution layer with stride 2 after a convolution layer

In [ ]:
nn = 2 #Number of neural networks (models)
model = [0] * nn

#Build model
for i in range(nn):
    model[i] = tf.keras.models.Sequential()
    model[i].add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
    model[i].add(BatchNormalization())
    model[i].add(Conv2D(32, (3, 3), activation = 'relu'))
    model[i].add(BatchNormalization())
    if(i == 0):
        model[i].add(MaxPooling2D(2, 2))
    else:
        model[i].add(Conv2D(32, (5, 5), strides = (2, 2), padding = 'same', activation = 'relu'))
    model[i].add(BatchNormalization())
    model[i].add(Dropout(0.4))
    
    model[i].add(Conv2D(64, (3, 3), activation = 'relu'))
    model[i].add(BatchNormalization())
    model[i].add(Conv2D(64, (3, 3), activation = 'relu'))
    model[i].add(BatchNormalization())
    if(i == 0):
        model[i].add(MaxPooling2D(2, 2))
    else:
        model[i].add(Conv2D(64, (5, 5), strides = (2, 2), padding = 'same', activation = 'relu'))
    model[i].add(BatchNormalization())
    model[i].add(Dropout(0.4))
    
    model[i].add(Flatten())
    model[i].add(Dense(128, activation = 'relu'))
    model[i].add(Dropout(0.4))
    model[i].add(Dense(10, activation = 'softmax'))
    
#Compile model
for i in range(nn):
    model[i].compile(loss = 'sparse_categorical_crossentropy',
                     optimizer = 'adam',
                     metrics = ['accuracy'])

#Summarize models
for i in range(nn):
    model[i].summary()

## Train the models

In [ ]:
history = [0] * nn
epochs = 10
steps_per_epoch = 50
for i in range(nn):
    history[i] = model[i].fit(x_train, y_train,
                              batch_size = 32,
                              steps_per_epoch = steps_per_epoch,
                              epochs = epochs,
                              validation_data = (x_val, y_val),
                              verbose = 1)
    print("Model {}, acc: {}, validation acc: {}".format(i, max(history[i].history['accuracy']), max(history[i].history['val_accuracy'])))

In [ ]:
acc_1 = history[0].history['accuracy']
vacc_1 = history[0].history['val_accuracy']
acc_2 = history[1].history['accuracy']
vacc_2 = history[1].history['val_accuracy']

# epochs = range(len(acc_1)) 
epochs = range(1, 11)

plt.plot(epochs, acc_1, 'b', label = 'Training Accuracy - Model 1')
plt.plot(epochs, vacc_1, 'b--', label = 'Validation Accuracy - Model 1')
plt.plot(epochs, acc_2, 'r', label = 'Training Accuracy - Model 2')
plt.plot(epochs, vacc_2, 'r--', label = 'Validation Accuracy - Model 2')

plt.title('Training and Validation accuracy')
plt.legend()
plt.show()
plt.savefig('./comparison_10.jpg')